In [1]:
import pandas as pd
import json
from web3 import Web3
from datetime import datetime, timedelta
import time
import sys
import json
from collections import Counter

# Connect to ETH blockchain with infura API key
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/5f0c4998d7544ee1bb3f0dc297a6821c'))

# AAVE V3 ETH Mainnet Market Pool Address
contract_addr = '0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2'

# Load ABI
with open("./abi.json") as f:
    pool_abi = json.load(f)
    
# Load Contract
aave_pool_contract = w3.eth.contract(address=contract_addr, abi=pool_abi)

In [2]:
def get_initial_depo_blockNum(aave_pool_contract, user_wallet_address):
    """
    Finds the lowest block number for rETH supply events related to a given user wallet address for AAVE v3.

    Parameters:
    - aave_pool_contract: The Aave pool contract instance from Web3.py.
    - user_wallet_address: The wallet address of the user as a string.

    Returns:
    - The lowest block number for the given user wallet address. Returns None if the user has no supply events.
    """
    # For some reason it doesnt work when user filter vs onBehalfOf
    log_filter = {
        'reserve': '0xae78736Cd615f374D3085123A210448E74Fc6393', #rETH token address
        'onBehalfOf': user_wallet_address,
    }
    
    logs = aave_pool_contract.events.Supply().get_logs(fromBlock='earliest',
                                                       toBlock='latest',
                                                       argument_filters=log_filter)
    if not logs:
        return 0
    
    return min(log['blockNumber'] for log in logs)

In [3]:
def fetch_logs(user_address_filter, aave_pool_contract, event_name, block_step = 100000):
    """
    Fetches logs for transactions filtered by user address and event type, from the latest block down to initial rETH deposit.

    Parameters:
    - user_address_filter: The user wallet address to filter the logs.
    - aave_pool_contract: The contract object to fetch logs from.
    - event_name: The name of the event to fetch {'Borrow', 'Repay', 'Withdraw', 'FlashLoan'}
    - block_step: The number of blocks to step back in each iteration (default is 100000).

    Returns:
    - A dataframe of formatted logs.
    """
    
    # Get target block
    target_block = get_initial_depo_blockNum(aave_pool_contract, user_address_filter)
    
    print(f'For User: {user_address_filter}\nFrom Block: {target_block}')

    # Dynamic print
    print(f'Fetching {event_name} events', end='')
    sys.stdout.flush()

    # Variables for looping
    logged_results = []
    target_log_count = float('inf')
    current_block = w3.eth.block_number    # Latest Block Number

    # Time
    start = time.time()

    # Loop counter
    iter_count = 0

    # Loop until target number reached or at first block
    while (len(logged_results) < target_log_count and current_block > target_block):
        print('.', end='')

        from_block = max(current_block - block_step, target_block)

        try:
            # Access event
            event = getattr(aave_pool_contract.events, event_name)()
            
            # Fetch logs for the current block
            if (event_name == 'FlashLoan'):
                current_logs = event.get_logs(
                    fromBlock=from_block,
                    toBlock=current_block,
                    argument_filters={'initiator': user_address_filter}
                )
            elif (event_name == 'Borrow'):
                current_logs = event.get_logs(
                    fromBlock=from_block,
                    toBlock=current_block,
                    argument_filters={'onBehalfOf': user_address_filter}
                )
            elif (event_name == 'Withdraw'):
                current_logs = event.get_logs(
                    fromBlock=from_block,
                    toBlock=current_block,
                    argument_filters={'to': user_address_filter}
                )
            else: #repay
                current_logs = event.get_logs(
                    fromBlock=from_block,
                    toBlock=current_block,
                    argument_filters={'user': user_address_filter}
                )

            # Format event logs
            for event in current_logs:
                formatted_log = format_event_log(event, event_name)

                # Add to aggregate log list
                logged_results.append(formatted_log)

            # Decrement block number
            current_block = from_block - 1

        except Exception as e:
            # Print exception and block number and break
            print(f"\nError fetching logs for block {current_block}: {e}")
            break

        # Increment iter
        iter_count += 1

    # Print time
    print('\nFetching complete.')
    print(f'Time Elapsed: {time.time() - start}\n')

    # Trim list
    if len(logged_results) > target_log_count:
           logged_results = logged_results[-target_log_count:]

    # Return dataframed logs
    return pd.DataFrame(logged_results)

def format_event_log(event, event_name):
    """
    Formats the log of an event based on its type.

    Parameters:
    - event: The event log to format.
    - event_name: The name of the event.

    Returns:
    - A dictionary containing formatted log attributes.
    """
    
    # Format common attrs
    log = {
        'Event Type': event_name,
        'Transaction Hash': event['transactionHash'].hex(),
        'Address': event['address'],
        'Block Hash': event['blockHash'].hex(),
        'Block Number': event['blockNumber'],
    }
    
    # if else tree for event specific attrs
    if event_name == 'Withdraw':
        log.update({
            'Reserve': event['args']['reserve'],
            'User': event['args']['user'],
            'To': event['args']['to'],
            'Amount': event['args']['amount']
        })
        
    elif event_name == 'Borrow':
        log.update({
            'Reserve': event['args']['reserve'],
            'On Behalf Of': event['args']['onBehalfOf'],
            'User': event['args']['user'],
            'Amount': event['args']['amount'],
            'Borrow Rate': event['args']['borrowRate']
        })
        
    elif event_name == 'Repay':
        log.update({
            'Reserve': event['args']['reserve'],
            'User': event['args']['user'],
            'Repayer': event['args']['repayer'],
            'Amount': event['args']['amount'],
            'useAtokens': str(event['args']['useATokens']),
            
        })
        
    elif event_name == 'FlashLoan':
        log.update({
            'Target': event['args']['target'],
            'Asset': event['args']['asset'],
            'Referral Code': str(event['args']['referralCode']),
            'Initiator': event['args']['initiator'],
            'Amount': event['args']['amount'],
            'Premium': event['args']['premium']
        })
    
    return log

In [4]:
# Map market name to pool contract address and abi filepath
contract_address_abi_map = {
    'AAVE': ('0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2', './abi.json')
    # fill with rest
}

def user_history(user_addresses, markets):
    """
    Fetches the interaction history of users with specified lending and borrowing markets.
    
    Parameters:
    - user_addresses: List of user wallet addresses.
    - markets: List of market names to scrape.
    
    Returns:
    - A dictionary mapping each user address to a list of dataframes (one for each interaction type).
    """
    interaction_history = {user: [] for user in user_addresses}
    event_types = ['Withdraw', 'Borrow', 'Repay', 'FlashLoan']
    
    # Loop markets
    for market in markets:
        # Lookup contract address and abi filepath
        contract_address, abi_filepath = contract_address_abi_map[market]
        
        # Web3 setup
        with open(abi_filepath) as f:
            abi = json.load(f)

        smart_contract = w3.eth.contract(address=contract_address, abi=abi)
        
        # Loop user addresses
        for user_address in user_addresses:
            # Loop event types
            for event_type in event_types:
                logs_df = fetch_logs(user_address, smart_contract, event_type, block_step = 250000)
                interaction_history[user_address].append(logs_df)
            
    return interaction_history
                

In [12]:
# TEST USER HISTORY

# Load wallet address data for testing
addresses_df = pd.read_csv('user_wallet_addresses.csv')
user_wallet_addresses = addresses_df['User Wallet Addresses'].tolist()

interaction_list = user_history(user_wallet_addresses, ['AAVE'])

For User: 0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB
From Block: 19099490
Fetching Withdraw events..
Fetching complete.
Time Elapsed: 0.112945556640625

For User: 0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB
From Block: 19099490
Fetching Borrow events..
Fetching complete.
Time Elapsed: 0.08099055290222168

For User: 0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB
From Block: 19099490
Fetching Repay events..
Fetching complete.
Time Elapsed: 0.07064151763916016

For User: 0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB
From Block: 19099490
Fetching FlashLoan events..
Fetching complete.
Time Elapsed: 1.6830947399139404

For User: 0x1A415A3b77ED38B683Db945dc83F53ECda26F716
From Block: 19099534
Fetching Withdraw events..
Fetching complete.
Time Elapsed: 0.08647990226745605

For User: 0x1A415A3b77ED38B683Db945dc83F53ECda26F716
From Block: 19099534
Fetching Borrow events..
Fetching complete.
Time Elapsed: 0.16286277770996094

For User: 0x1A415A3b77ED38B683Db945dc83F53ECda26F716
From Block: 19099534
Fetch

Fetching FlashLoan events......
Fetching complete.
Time Elapsed: 5.973413944244385

For User: 0x78e96Be52e38b3FC3445A2ED34a6e586fFAb9631
From Block: 18489769
Fetching Withdraw events....
Fetching complete.
Time Elapsed: 0.36112284660339355

For User: 0x78e96Be52e38b3FC3445A2ED34a6e586fFAb9631
From Block: 18489769
Fetching Borrow events....
Fetching complete.
Time Elapsed: 0.19483685493469238

For User: 0x78e96Be52e38b3FC3445A2ED34a6e586fFAb9631
From Block: 18489769
Fetching Repay events....
Fetching complete.
Time Elapsed: 0.15902423858642578

For User: 0x78e96Be52e38b3FC3445A2ED34a6e586fFAb9631
From Block: 18489769
Fetching FlashLoan events....
Fetching complete.
Time Elapsed: 4.419696569442749

For User: 0x5E01C1d9f0198Ac4f02696e0A12CfA7969d172Da
From Block: 0
Fetching Withdraw events..............................................................................
Fetching complete.
Time Elapsed: 3.705087184906006

For User: 0x5E01C1d9f0198Ac4f02696e0A12CfA7969d172Da
From Block: 0
Fetch

For User: 0x660Ec6b32C58f682e5d5C619758440e1357bD58C
From Block: 18879342
Fetching Borrow events...
Fetching complete.
Time Elapsed: 0.2071225643157959

For User: 0x660Ec6b32C58f682e5d5C619758440e1357bD58C
From Block: 18879342
Fetching Repay events...
Fetching complete.
Time Elapsed: 0.16970014572143555

For User: 0x660Ec6b32C58f682e5d5C619758440e1357bD58C
From Block: 18879342
Fetching FlashLoan events...
Fetching complete.
Time Elapsed: 2.5115721225738525

For User: 0xB58EBb9e37EBdd66eDF76499B6D4044175bA0484
From Block: 19192598
Fetching Withdraw events.
Fetching complete.
Time Elapsed: 0.04250025749206543

For User: 0xB58EBb9e37EBdd66eDF76499B6D4044175bA0484
From Block: 19192598
Fetching Borrow events.
Fetching complete.
Time Elapsed: 0.03683781623840332

For User: 0xB58EBb9e37EBdd66eDF76499B6D4044175bA0484
From Block: 19192598
Fetching Repay events.
Fetching complete.
Time Elapsed: 0.05235576629638672

For User: 0xB58EBb9e37EBdd66eDF76499B6D4044175bA0484
From Block: 19192598
Fetchin

For User: 0x0dCf166dDaF37C44A55F868A52B63d3Be1D51669
From Block: 19224015
Fetching Repay events.
Fetching complete.
Time Elapsed: 0.04227590560913086

For User: 0x0dCf166dDaF37C44A55F868A52B63d3Be1D51669
From Block: 19224015
Fetching FlashLoan events.
Fetching complete.
Time Elapsed: 0.9343211650848389

For User: 0xB2df2d124cC5a4C9E3eaeB9719CE2DCEC88e9194
From Block: 18089678
Fetching Withdraw events......
Fetching complete.
Time Elapsed: 0.26015233993530273

For User: 0xB2df2d124cC5a4C9E3eaeB9719CE2DCEC88e9194
From Block: 18089678
Fetching Borrow events......
Fetching complete.
Time Elapsed: 0.265627384185791

For User: 0xB2df2d124cC5a4C9E3eaeB9719CE2DCEC88e9194
From Block: 18089678
Fetching Repay events......
Fetching complete.
Time Elapsed: 0.2226884365081787

For User: 0xB2df2d124cC5a4C9E3eaeB9719CE2DCEC88e9194
From Block: 18089678
Fetching FlashLoan events......
Fetching complete.
Time Elapsed: 6.303126096725464

For User: 0x74B70c07d64D5d4EcbEA925EC66c2eA0D5B6a67f
From Block: 192

Fetching FlashLoan events.
Fetching complete.
Time Elapsed: 0.8828656673431396

For User: 0xf5dB697A5aD188FDDc9258FFF2206253932A467D
From Block: 18783909
Fetching Withdraw events...
Fetching complete.
Time Elapsed: 0.13272309303283691

For User: 0xf5dB697A5aD188FDDc9258FFF2206253932A467D
From Block: 18783909
Fetching Borrow events...
Fetching complete.
Time Elapsed: 0.13799619674682617

For User: 0xf5dB697A5aD188FDDc9258FFF2206253932A467D
From Block: 18783909
Fetching Repay events...
Fetching complete.
Time Elapsed: 0.14848589897155762

For User: 0xf5dB697A5aD188FDDc9258FFF2206253932A467D
From Block: 18783909
Fetching FlashLoan events...
Fetching complete.
Time Elapsed: 3.1475675106048584

For User: 0x551C5aAdB287D086B47DaBd0cf4d8e83965e79B5
From Block: 18469732
Fetching Withdraw events....
Fetching complete.
Time Elapsed: 0.16596055030822754

For User: 0x551C5aAdB287D086B47DaBd0cf4d8e83965e79B5
From Block: 18469732
Fetching Borrow events....
Fetching complete.
Time Elapsed: 0.21140766

In [15]:
all_dfs = []

for key, dfs_list in interaction_list.items():
    # Concatenate all DataFrames in the list for the current wallet address
    combined_df = pd.concat(dfs_list, ignore_index=True)
    all_dfs.append(combined_df)

# Concatenate all combined DataFrames into one
final_df = pd.concat(all_dfs, ignore_index=True)
final_df

,Event Type,Transaction Hash,Address,Block Hash,Block Number,Reserve,On Behalf Of,User,Amount,Borrow Rate,Repayer,useAtokens,To,Target,Asset,Referral Code,Initiator,Premium
0,Borrow,0x922675f27fd9a372bedb978dcdffd27d60631667b9f5...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x395cf3cad6c726e6e7cefef81dab65c1ee9f0cff8c63...,19253214,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,50000000000,90557895041748529955456040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Borrow,0xa9b6bdde432cce8a7877e943161b3c3fc302691b0bdc...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x0aebf5f1b8237ed5174836af0e81a9bb7b9d21151a5e...,19395873,0xdAC17F958D2ee523a2206206994597C13D831ec7,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,150000000000,104111745662285810140385993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Repay,0x7502a548129b945b074fbc3143e693dcc84509342d05...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x6a5b9a35929e4c764697273b609e58045667ffd51902...,19194122,0xdAC17F958D2ee523a2206206994597C13D831ec7,NaN,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,15514000000,NaN,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,False,NaN,NaN,NaN,NaN,NaN,NaN
3,Repay,0xc5532707ea840d9fb1e8acad45f8cb3d31e25839b216...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x86cbfa0fcae8734577aa57710fb2d0b181dd9d438f7e...,19194127,0xdAC17F958D2ee523a2206206994597C13D831ec7,NaN,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,139623000000,NaN,0xc83894aC498289E54FBE523c0ABFcbac8CeB1CaB,False,NaN,NaN,NaN,NaN,NaN,NaN
4,Borrow,0x53237b478e01073f714423746b4947b3fe3c70906f57...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0xc208dcc0d793e304440c7ac1e21f9bd39bf6479a3a25...,19223002,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0x1A415A3b77ED38B683Db945dc83F53ECda26F716,0x1A415A3b77ED38B683Db945dc83F53ECda26F716,12900000000,115909922884666592392623660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13714,Withdraw,0x92765195b71326a11cc2917275a5b6ce03bbb04f836b...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x8a89fbec2c17524589c1678683d1c3c0525f674be43d...,19352043,0xae78736Cd615f374D3085123A210448E74Fc6393,NaN,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,201000000000000000000,NaN,NaN,NaN,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,NaN,NaN,NaN,NaN,NaN
13715,Borrow,0xf29a024f48cafc84aa26cb27010676d8bf46562c56d1...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x68e759b4c48ed73c06cc85bf294d5ad3d7cb83f5d19b...,19301585,0xdAC17F958D2ee523a2206206994597C13D831ec7,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,670000000000,59387665988109620450038412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13716,Borrow,0xd4dddbaf889488dd6f955b04c3df948988ae01772c16...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0xd5c1822d197346f6309ee693e8c89ccf3127e66b6419...,19080025,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,250000000000,59622078712477660230987677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13717,Repay,0x5fefe317248cdd1dbd837f81db17e8f0b44930b5e5ae...,0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2,0x27df8ea99c1db9df10d23a795ac325db5b7a51969bcc...,19228650,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,NaN,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,251108520091,NaN,0xa7F3C74f0255796Fd5D3DDCf88db769f7a6bf46a,False,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
final_df.to_csv('./transaction_data.csv', index=False)